In [2]:
import cv2
import numpy as np

In [3]:
# YOLO için gerekli dosyaların ve konfigürasyonun yolu
yolo_weights = "yolov3.weights"
yolo_cfg = "yolov3.cfg"

# YOLO ile eğitilmiş modeli yükleme
net = cv2.dnn.readNet(yolo_weights, yolo_cfg)

# Sınıf etiketlerini yükleme
classes = []
with open("walking.mp4", "r", encoding="latin-1") as f:
    classes = f.read().strip().split("\n")


# YOLO'nun çıkış katmanlarını alma
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Video dosyasını açma
video_path = r'C:\Users\User\Desktop\Face Recognition\Events\x2mate.com-Okulöncesi .mp4'
cap = cv2.VideoCapture(video_path)

while True:
    # Video çerçevesini okuma
    ret, frame = cap.read()
    if not ret:
        break

    # Video çerçevesini YOLO ile işleme alma
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # YOLO'dan gelen çıktıları işleme alma
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # 0 indeksi insan sınıfını temsil eder
                # Nesne algılama kutusunun koordinatlarını alma
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Kutunun köşe koordinatlarını hesaplama
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-maximum suppression uygulama
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Algılanan yüzleri çerçeve üzerine çizme ve kaydetme
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)  # Yeşil renk
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 5), font, 1, color, 1)

            # Algılanan yüzleri başka bir klasöre kaydetme
            face = frame[y:y+h, x:x+w]
            cv2.imwrite("output_faces/face{}.jpg".format(i), face)

    # Sonuçları ekranda gösterme
    cv2.imshow("YOLO Face Detection", frame)

    # Çıkış için 'q' tuşuna basılmasını bekleyin
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Temizlik ve çıkış
cap.release()
cv2.destroyAllWindows()

IndexError: invalid index to scalar variable.